In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

test_data = pd.read_csv("/kaggle/input/titanic/test.csv")



df = pd.read_csv('/kaggle/input/titanic/train.csv')

X = df.drop('Survived', axis=1)

y = df.Survived

drops = ['Name','Ticket']
test_data = test_data.drop(drops, axis=1)
X = X.drop(drops, axis=1)
X







,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,female,38.0,1,0,71.2833,C85,C
2,3,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,female,35.0,1,0,53.1000,C123,S
4,5,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,NaN,S
887,888,1,female,19.0,0,0,30.0000,B42,S
888,889,3,female,NaN,1,2,23.4500,NaN,S
889,890,1,male,26.0,0,0,30.0000,C148,C


In [2]:
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'), ['Sex', 'Cabin', 'Embarked']),
    remainder='passthrough')
transformed = transformer.fit_transform(X).toarray()

X = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names()
)

transformed = transformer.transform(test_data).toarray()
test_data = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names()
)

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(X)
X = imp_mean.transform(X)
X = pd.DataFrame(X,columns=imp_mean.feature_names_in_)
X = X.astype({'PassengerId': 'int32'})

test_data = imp_mean.transform(test_data)
test_data = pd.DataFrame(test_data,columns=imp_mean.feature_names_in_)
test_data = test_data.astype({'PassengerId': 'int32'})

test_data.dtypes

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


onehotencoder__x0_female    float64
onehotencoder__x0_male      float64
onehotencoder__x1_A10       float64
onehotencoder__x1_A14       float64
onehotencoder__x1_A16       float64
                             ...   
Pclass                      float64
Age                         float64
SibSp                       float64
Parch                       float64
Fare                        float64
Length: 160, dtype: object

In [3]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)

clf = RandomForestClassifier(n_estimators=200, max_depth=4, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.78      0.99      0.87       211
           1       0.97      0.60      0.74       146

    accuracy                           0.83       357
   macro avg       0.87      0.79      0.80       357
weighted avg       0.86      0.83      0.82       357



In [4]:
predictions = clf.predict(test_data)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
